In [1]:
%env TOKENIZERS_PARALLELISM=false
!pip install --quiet keybert
print('pip install keybert complete')

env: TOKENIZERS_PARALLELISM=false
pip install keybert complete


In [2]:
from urllib import request
url = 'https://www.gutenberg.org/cache/epub/17969/pg17969.txt'
lines = [line.decode('utf-8') for line in request.urlopen(url=url)]
print(len(lines))
text = ' '.join(lines).replace('\n', ' ').replace('\r', ' ')
text = ' '.join(text.split())


1024


In [3]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = list(tokenizer.tokenize(text))
print(len(sentences))

365


In [4]:
# remove boilerplate text
sentences = sentences[13:-120]

In [5]:
from arrow import now
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

MIN_DF = 3
MODEL = 'all-MiniLM-L12-v2'
STOP_WORDS = 'english'

model_start = now()
model = KeyBERT(model=MODEL,)
# model.max_seq_length = 512
vectorizer = CountVectorizer(ngram_range=(1, 2), stop_words=STOP_WORDS, min_df=MIN_DF,)
document_embeddings, word_embeddings = model.extract_embeddings(docs=sentences, vectorizer=vectorizer, )
print('embedding time: {}'.format(now() - model_start))
print('we have {} documents and {} words.'.format(len(document_embeddings), len(word_embeddings)))
keywords = model.extract_keywords(docs=sentences, top_n=1, stop_words=STOP_WORDS, vectorizer=vectorizer,
                                  doc_embeddings=document_embeddings, word_embeddings=word_embeddings, min_df=MIN_DF, )
print('model time: {}'.format(now() - model_start))

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

embedding time: 0:00:10.048351
we have 232 documents and 247 words.
model time: 0:00:10.218848


In [6]:
import pandas as pd
from plotly.express import scatter
from umap import UMAP

umap_start = now()
umap_model = UMAP(n_components=2, random_state=2024, verbose=False, n_jobs=1)
df = pd.DataFrame(data=umap_model.fit_transform(X=document_embeddings), columns=['u0', 'u1'])
df['text'] = sentences
df['short text'] = [' '.join(item.split()[:20]) for item in sentences]
df['keyword'] = [keyword[0][0] if len(keyword) else '-none-' for keyword in keywords]
IGNORE = {'-none-', 'section', 'paragraph'}
scatter(data_frame=df[~df['keyword'].isin(IGNORE)], x='u0', y='u1', hover_name='short text',
        height=900, hover_data=['keyword']
       ).show()
print('UMAP time: {}'.format(now() - umap_start))

UMAP time: 0:00:11.952906


In [7]:
print(df['keyword'].value_counts().to_dict())

{'mounds': 17, 'mound builders': 12, 'altar': 10, 'skeletons': 8, '-none-': 6, 'acres': 6, 'burial mounds': 6, 'burial': 6, 'serpent': 5, 'mound': 5, 'ohio': 4, 'trees': 4, 'valleys': 4, 'mining': 4, 'enclosure': 3, 'fortification': 3, 'mounds_': 3, 'ancient race': 3, 'skeleton': 3, 'sacred enclosures': 3, 'county ohio': 3, 'tribes': 3, 'prehistoric': 3, 'ornaments': 3, 'civilized': 3, 'wall': 3, 'avenue': 2, 'length': 2, 'buried': 2, 'signal mound': 2, 'temples': 2, 'journey': 2, 'soil': 2, 'copper': 2, 'hebrew': 2, 'indian tribes': 2, 'readings': 2, 'valley': 2, 'elevation': 2, 'antiquity': 2, 'parallel walls': 2, 'mysterious': 2, 'race': 2, 'east': 2, 'civilization': 2, 'existence': 1, 'south': 1, 'river': 1, 'wisconsin': 1, 'mexico': 1, 'grave': 1, 'numbers': 1, 'animals': 1, 'north': 1, 'indians': 1, 'height': 1, 'shape': 1, 'alligator': 1, 'ancient': 1, 'places': 1, 'sacred': 1, 'animal': 1, 'structure': 1, 'evidences': 1, 'ago': 1, 'relics': 1, 'implements': 1, 'course': 1, 'age

In [8]:
import numpy as np
from plotly.express import scatter
from plotly.graph_objects import Figure

def plot_words(arg_words: list, arg_keywords: list, arg_model: UMAP, arg_embeddings: np.ndarray) -> Figure:
    top_indices = [arg_words.tolist().index(keyword) for keyword in arg_keywords]
    result_df = pd.DataFrame(data=arg_model.transform(X=[arg_embeddings[index] for index in top_indices]), 
                             columns=['u0', 'u1'])
    result_df['word'] = arg_keywords
    return scatter(data_frame=result_df, x='u0', y='u1', text='word', height=900).update_traces(marker={'size': 3})

plot_words(arg_words=vectorizer.get_feature_names_out(),
           arg_keywords=[item for item in df['keyword'].unique().tolist() if item != '-none-'],
           arg_model=umap_model, arg_embeddings=word_embeddings).show()